In [2]:
import numpy as np
import pandas as pd
from typing import List
# from numba import jit
# import numba

np.seterr(divide='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [7]:
def simplex_max(table: np.ndarray, columns: List[str] = None, rows: List[str] = None) -> dict:
    iteration = 0
    columns = ['free'] + [f'X{i}' for i in range(1, table.shape[1])] if columns is None else columns
    rows = [f'X{i + table.shape[1]}' for i in range(table.shape[0] - 1)] + ['F'] if rows is None else rows
    table = np.array(table).copy().astype(float)
    leading_col = None

    start_table = pd.DataFrame(table, columns=columns, index=rows)

    print("iteration:", iteration)
    print(start_table)

    while np.sum(table[-1] >= 0) < table.shape[1]:
        iteration += 1
        print("iteration:", iteration)

        table2 = table.copy()
        # if leading_col is not None:
        #     table2[0, leading_col] = 0

        # leading_col = abs(table2[:-1, 1:]).argmax() % (table.shape[1] - 1) + 1 \
        #     if sum(table2[-1] <= 0) > 1 or table2[-1, 0] < 0 else np.where(table2[-1] <= 0)[0][0]
        leading_col = np.where(table2[-1, 1:] <= 0)[0][0] + 1 if sum(table2[-1, 1:] <= 0) == 1 \
            else np.abs(table2[:-1, 1:]).argmax() % (table.shape[1] - 1) + 1

        col = np.divide(table2.T[0], table2.T[leading_col])
        col[np.logical_and(table2.T[0] >= 0, table2.T[leading_col] < 0)] = np.inf
        # if sum(np.isinf(col)) == table.shape[0]:
        #     print(table)
        #     print(table2)
        #     print(leading_col)
        #     print(col)
        #     raise Exception
        leading_row = np.abs(col).argmin()

        # print(table)
        # print(col)
        # print(leading_col, leading_row)

        table2[0] = table[leading_row] / table[leading_row, leading_col]
        table2[0, leading_col] = 1 / table[leading_row, leading_col]
        auxiliary = -table.T[leading_col]
        auxiliary = np.delete(auxiliary, leading_row)
        table = np.delete(table, leading_row, axis=0)
        table.T[leading_col] = 0

        for i in range(auxiliary.shape[0]):
            table2[i + 1] = table2[0] * auxiliary[i] + table[i]

        table = table2

        changed_col = columns[leading_col]
        columns[leading_col] = rows.pop(leading_row)
        rows.insert(0, changed_col)

        print("leading_col:", leading_col)
        print(pd.DataFrame(table, columns=columns, index=rows))
        print(auxiliary)
        # input()

    values = pd.DataFrame(table, columns=columns, index=rows).iloc[:, 0].to_dict()
    for i in range(1, len(columns)):
        values[columns[i]] = 0
    return values


simplex_max(np.array([
    [-2, 1, -2],
    [-4, -1, -1],
    [2, 1, -1],
    [6, 0, 1],
    [0, -1, -2],
]).astype(float))

iteration: 0
    free   X1   X2
X3  -2.0  1.0 -2.0
X4  -4.0 -1.0 -1.0
X5   2.0  1.0 -1.0
X6   6.0  0.0  1.0
F    0.0 -1.0 -2.0
iteration: 1
leading_col: 2
    free   X1   X3
X2   1.0 -0.5 -0.5
X4  -3.0 -1.5 -0.5
X5   3.0  0.5 -0.5
X6   5.0  0.5  0.5
F    2.0 -2.0 -1.0
[ 1.  1. -1.  2.]
iteration: 2
leading_col: 1
    free        X4        X3
X1   2.0 -0.666667  0.333333
X2   2.0 -0.333333 -0.333333
X5   2.0  0.333333 -0.666667
X6   4.0  0.333333  0.333333
F    6.0 -1.333333 -0.333333
[ 0.5 -0.5 -0.5  2. ]
iteration: 3
leading_col: 1
    free   X5   X3
X4   6.0  3.0 -2.0
X1   6.0  2.0 -1.0
X2   4.0  1.0 -1.0
X6   2.0 -1.0  1.0
F   14.0  4.0 -3.0
[ 0.66666667  0.33333333 -0.33333333  1.33333333]
iteration: 4
leading_col: 2
    free   X5   X6
X3   2.0 -1.0  1.0
X4  10.0  1.0  2.0
X1   8.0  1.0  1.0
X2   6.0  0.0  1.0
F   20.0  1.0  3.0
[2. 1. 1. 3.]


{'X3': 2.0, 'X4': 10.0, 'X1': 8.0, 'X2': 6.0, 'F': 20.0, 'X5': 0, 'X6': 0}

sum(Ui[0] * Xi) -> max

sum(U[y] * Xi) - R[y] >= 0

X > 0

U1 * X1 + U2 * X2 + U3 * X3 -> max

U1z * X1 + U2z * X2 + U3z * X3 <= Rz

U1d * X1 + U2d * X2 + U3d * X3 <= Rd

U1k * X1 + U2k * X2 + U3k * X3 <= Rk

U1z * X1 + U2z * X2 + U3z * X3 + Z = Rz

U1d * X1 + U2d * X2 + U3d * X3 + D = Rd

U1k * X1 + U2k * X2 + U3k * X3 + K = Rk

F = 0 - (-U1 * X1 - U2 * X2 - U3 * X3)

Z = Rz - (U1z * X1 + U2z * X2 + U3z * X3)

D = Rd - (-U1d * X1 + U2d * X2 + U3d * X3)

K = Rk - (-U1k * X1 + U2k * X2 + U3k * X3)

In [8]:
U = {
    (4, 'Маг'): {
        'Z': 5,
        'D': 3,
        'K': 4,
    },
    (5, 'Рыцарь'): {
        'Z': 4,
        'D': 5,
        'K': 7,
    },
    (200, 'Дракон'): {
        'Z': 50,
        'D': 35,
        'K': 40,
    }
}

R = {
    'Z': 400,
    'D': 300,
    'K': 500,
}

In [9]:
Xs = [[Rv] + [Uv[Rk] for Uk, Uv in U.items()] for Rk, Rv in R.items()] + [[0] + [-Uk[0] for Uk, Uv in U.items()]]
np.array(Xs)

array([[ 400,    5,    4,   50],
       [ 300,    3,    5,   35],
       [ 500,    4,    7,   40],
       [   0,   -4,   -5, -200]])

In [10]:
simplex_max(Xs, rows=['Z', 'D', 'K', 'F'], columns=['free', 'X1', 'X2', 'X3'])

iteration: 0
    free   X1   X2     X3
Z  400.0  5.0  4.0   50.0
D  300.0  3.0  5.0   35.0
K  500.0  4.0  7.0   40.0
F    0.0 -4.0 -5.0 -200.0
iteration: 1
leading_col: 3
      free    X1     X2     Z
X3     8.0   0.1   0.08  0.02
D     20.0  -0.5   2.20 -0.70
K    180.0   0.0   3.80 -0.80
F   1600.0  16.0  11.00  4.00
[-35. -40. 200.]


{'X3': 8.0, 'D': 20.0, 'K': 180.0, 'F': 1600.0, 'X1': 0, 'X2': 0, 'Z': 0}

In [15]:
U = {
    (4, 'Маг'): {
        'Z': 5,
        'D': 3,
        'K': 4,
    },
    (5, 'Рыцарь'): {
        'Z': 4,
        'D': 5,
        'K': 7,
    },
    (50, 'Дракон'): {
        'Z': 50,
        'D': 35,
        'K': 45,
    }
}

R = {
    'Z': 400,
    'D': 300,
    'K': 500,
}

In [16]:
Xs = [[Rv] + [Uv[Rk] for Uk, Uv in U.items()] for Rk, Rv in R.items()] + [[0] + [-Uk[0] for Uk, Uv in U.items()]]
np.array(Xs)

array([[400,   5,   4,  50],
       [300,   3,   5,  35],
       [500,   4,   7,  45],
       [  0,  -4,  -5, -50]])

In [17]:
simplex_max(Xs, rows=['Z', 'D', 'K', 'F'], columns=['free', 'X1', 'X2', 'X3'])

iteration: 0
    free   X1   X2    X3
Z  400.0  5.0  4.0  50.0
D  300.0  3.0  5.0  35.0
K  500.0  4.0  7.0  45.0
F    0.0 -4.0 -5.0 -50.0
iteration: 1
leading_col: 3
     free   X1    X2     Z
X3    8.0  0.1  0.08  0.02
D    20.0 -0.5  2.20 -0.70
K   140.0 -0.5  3.40 -0.90
F   400.0  1.0 -1.00  1.00
[-35. -45.  50.]
iteration: 2
leading_col: 2
          free        X1         D         Z
X2    9.090909 -0.227273  0.454545 -0.318182
X3    7.272727  0.118182 -0.036364  0.045455
K   109.090909  0.272727 -1.545455  0.181818
F   409.090909  0.772727  0.454545  0.681818
[-0.08 -3.4   1.  ]


{'X2': 9.090909090909092,
 'X3': 7.2727272727272725,
 'K': 109.0909090909091,
 'F': 409.09090909090907,
 'X1': 0,
 'D': 0,
 'Z': 0}

In [4]:
U = {
    (10, 'Маг'): {
        'Z': 5,
        'D': 1,
        'K': 1,
    },
    (5, 'Рыцарь'): {
        'Z': 4,
        'D': 5,
        'K': 7,
    },
    (200, 'Дракон'): {
        'Z': 0,
        'D': 35,
        'K': 40,
    }
}

R = {
    'Z': 400,
    'D': 300,
    'K': 500,
}
Xs = [[Rv] + [Uv[Rk] for Uk, Uv in U.items()] for Rk, Rv in R.items()] + [[0] + [-Uk[0] for Uk, Uv in U.items()]]
np.array(Xs)
simplex_max(Xs, rows=['Z', 'D', 'K', 'F'], columns=['free', 'X1', 'X2', 'X3'])

iteration: 0
    free    X1   X2     X3
Z  400.0   5.0  4.0    0.0
D  300.0   1.0  5.0   35.0
K  500.0   1.0  7.0   40.0
F    0.0 -10.0 -5.0 -200.0
iteration: 1
leading_col: 3
           free        X1         X2         D
X3     8.571429  0.028571   0.142857  0.028571
Z    400.000000  5.000000   4.000000  0.000000
K    157.142857 -0.142857   1.285714 -1.142857
F   1714.285714 -4.285714  23.571429  5.714286
[ -0. -40. 200.]
iteration: 2
leading_col: 1
           free         Z     X2         D
X1    80.000000  0.200000   0.80  0.000000
X3     6.285714 -0.005714   0.12  0.028571
K    168.571429  0.028571   1.40 -1.142857
F   2057.142857  0.857143  27.00  5.714286
[-0.02857143  0.14285714  4.28571429]


{'X1': 80.0,
 'X3': 6.285714285714286,
 'K': 168.57142857142858,
 'F': 2057.142857142857,
 'Z': 0,
 'X2': 0,
 'D': 0}